# sliding-puzzle

O *sliding-puzzle* é um quebra-cabeça deslizante que consiste em uma grade $N\times N$ na qual $N^2-1$ peças numeradas são colocadas aleatoriamente em $N^2-1$ das $N^2$ posições e uma posição está vazia, permitindo que as peças adjacentes sejam movidas para ocupar o espaço vazio, com o objetivo final de reorganizar as peças em uma ordem numérica específica, tipicamente de 1 a $N-1$, seguindo um número mínimo de movimentos.

Observe que em $N\times N$ posições podemos permutar as $N^2$ peças (contando o espaço vazio como peça), isto significa que temos $(N^2)!$ combinações possíveis. No entanto, nem todas essas permutações são possíveis no sliding-puzzle, pois algumas configurações não são alcançáveis a partir de outras por meio dos movimentos, é possível demonstrar que este jogo tem $\frac{(N^2)!}{2}$ estados possíveis. No caso $N=3$ temos 181440 combinações.

Nos exemplos a seguir utilizaremos os casos $N=2$ e $N=3$.

Utilizaremos este jogo para exemplificar algumas das técnicas de busca cega e heurística.

## Definição da classe sliding-puzzle

A classe definida a seguir é uma estrutura de dados que encapsula as funcionalidades necessárias para representar e manipular um quebra-cabeça sliding-puzzle. Observe que esta implementação é genérica, admitindo quebra-cabeças com $N$ qualquer definido pela matriz que é passada no construtor.

In [1]:
class SlidingPuzzle:
    def __init__(self, initial_state):
        tmp = tuple(map(tuple, initial_state))
        mutable_state  = [list(row) for row in tmp]
        self.state = mutable_state 
        if len(self.state)!=len(self.state[0]):
            raise ValueError("O estado inicial é inválido pois não é uma matriz quadrada.")
        self.N = len(self.state)

    def __str__(self):
        str_rep = ""
        for row in self.state:
            str_rep += " ".join(map(str, row)) + "\n"
        return str_rep
    
    def move(self, direction):
        # Encontra a posição do espaço vazio
        empty_pos = self.find_empty()

        # Verifica a direção e troca a posição do espaço vazio com a peça adjacente na direção especificada
        if direction == 'up' and empty_pos[0] > 0:
            self.state[empty_pos[0]][empty_pos[1]], self.state[empty_pos[0]-1][empty_pos[1]] = \
                self.state[empty_pos[0]-1][empty_pos[1]], self.state[empty_pos[0]][empty_pos[1]]
        elif direction == 'down' and empty_pos[0] < self.N-1:
            self.state[empty_pos[0]][empty_pos[1]], self.state[empty_pos[0]+1][empty_pos[1]] = \
                self.state[empty_pos[0]+1][empty_pos[1]], self.state[empty_pos[0]][empty_pos[1]]
        elif direction == 'left' and empty_pos[1] > 0:
            self.state[empty_pos[0]][empty_pos[1]], self.state[empty_pos[0]][empty_pos[1]-1] = \
                self.state[empty_pos[0]][empty_pos[1]-1], self.state[empty_pos[0]][empty_pos[1]]
        elif direction == 'right' and empty_pos[1] < self.N-1:
            self.state[empty_pos[0]][empty_pos[1]], self.state[empty_pos[0]][empty_pos[1]+1] = \
                self.state[empty_pos[0]][empty_pos[1]+1], self.state[empty_pos[0]][empty_pos[1]]

    def get_state(self):
        return tuple(map(tuple, self.state))
    
    def set_state(self, new_state):
        mutable_state  = [list(row) for row in new_state]
        self.state = mutable_state

    def find_empty(self):
        for i in range(self.N):
            for j in range(self.N):
                if self.state[i][j] == 0:
                    return (i, j)
                
    # Busca possíveis movimentos com possibilidade de descartar o ultimo movimento realizado
    def get_possible_moves(self, last_move='none'):
        possible_moves = []
        empty_pos = self.find_empty()
        if empty_pos[0] > 0 and last_move!='down':
            possible_moves.append('up')
        if empty_pos[0] < self.N-1 and last_move!='up':
            possible_moves.append('down')
        if empty_pos[1] > 0 and last_move!='right':
            possible_moves.append('left')
        if empty_pos[1] < self.N-1 and last_move!='left':
            possible_moves.append('right')
        return possible_moves
    
    def reverse_movement(self, move):
        if move == 'up':
            return 'down'
        if move == 'down':
            return 'up'
        if move == 'left':
            return 'right'
        if move == 'right':
            return 'left'
        return None
    
    def undo_move(self, move):
        self.move(self.reverse_movement(move))

Exemplo de uso da classe Puzzle8:

In [2]:
# Exemplo de uso
initial_state = [
    [1, 2, 3],
    [4, 0, 5],
    [6, 7, 8]
]

puzzle = SlidingPuzzle(initial_state)
print("Estado inicial:")
print(puzzle)

# Movendo a peça vazia para cima
puzzle.move('up')
print("Depois de mover para cima:")
print(puzzle)
puzzle.move('down')
print("Depois de mover para baixo:")
print(puzzle)
puzzle.move('right')
print("Depois de mover para direita:")
print(puzzle)
puzzle.move('left')
print("Depois de mover para esquerda:")
print(puzzle)

Estado inicial:
1 2 3
4 0 5
6 7 8

Depois de mover para cima:
1 0 3
4 2 5
6 7 8

Depois de mover para baixo:
1 2 3
4 0 5
6 7 8

Depois de mover para direita:
1 2 3
4 5 0
6 7 8

Depois de mover para esquerda:
1 2 3
4 0 5
6 7 8



## Busca Cega

### Backtracking

In [3]:
class BacktrackingSolver:
    def __init__(self, puzzle, goal):
        self.puzzle = puzzle
        self.visited = set()
        self.goal = goal

    def solve(self):
        # A solução inicializa com uma lista vazia de movimentos
        solution = self.backtrack()
        return solution

    # Função de busca por força bruta
    def backtrack(self, last_move='none', moves=[]):
        state = self.puzzle.get_state()
        
        if self.is_goal(state):
            return moves
        
        for move in self.puzzle.get_possible_moves(last_move):
            self.puzzle.move(move)
            
            solution = self.backtrack(move, moves + [move])
            
            if solution:
                return solution
            else:
                self.puzzle.undo_move(move)
                    
        return None

    # Função verifica estado final
    def is_goal(self, state):
        state_list = [list(row) for row in state]
        return state_list == self.goal

In [4]:
# Exemplo de uso
initial_state = [[3,2], [0,1]]
goal_state = [[0,1],[2,3]]
puzzle = SlidingPuzzle(initial_state)
solver = BacktrackingSolver(puzzle, goal_state)
solution = solver.solve()

if solution:
    print("Solução encontrada:")
    print(solution)
else:
    print("Não foi possível encontrar uma solução.")


Solução encontrada:
['up', 'right', 'down', 'left', 'up']


### Busca em Grafos

#### Busca em Profundidade

In [5]:
class DfsSolver:
    def __init__(self, puzzle, goal):
        self.puzzle = puzzle
        self.visited = set()
        self.goal = goal

    def solve(self):
        return self.dfs(self.puzzle.get_state())

    def dfs(self, state):
        if self.is_goal(state):
            return []

        self.visited.add(state)
        
        for move in self.puzzle.get_possible_moves():
            self.puzzle.move(move)
            
            next_state = self.puzzle.get_state()
            if next_state not in self.visited:
                solution = self.dfs(next_state)
                if solution is not None:
                    return [move] + solution
                
            self.puzzle.undo_move(move)

        return None
    
    # Função verifica estado final
    def is_goal(self, state):
        state_list = [list(row) for row in state]
        return state_list == self.goal

In [6]:
# Testando o solucionador
initial_state = [[3,2], [0,1]]
goal_state = [[0,1],[2,3]]
puzzle = SlidingPuzzle(initial_state)
solver = DfsSolver(puzzle, goal_state)
solution = solver.solve()

if solution:
    print("Solução encontrada:")
    print(solution)
else:
    print("Não foi possível encontrar uma solução.")


Solução encontrada:
['up', 'right', 'down', 'left', 'up']


#### Busca em Largura

In [7]:
from collections import deque

class BfsSolver:
    def __init__(self, puzzle, goal):
        self.puzzle = puzzle
        self.goal = goal

    def solve(self):
        queue = deque([(self.puzzle.get_state(), [])])
        visited = set()

        while queue:
            state, path = queue.popleft()

            if self.is_goal(state):
                return path

            if state in visited:
                continue
            visited.add(state)
            
            state_list = [list(row) for row in state]
            new_puzzle = SlidingPuzzle(state_list)

            for move in new_puzzle.get_possible_moves():
                new_puzzle.move(move)
                next_state = new_puzzle.get_state()
                queue.append((next_state, path + [move]))
                new_puzzle.undo_move(move)

        return None
    
    # Função verifica estado final
    def is_goal(self, state):
        state_list = [list(row) for row in state]
        return state_list == self.goal

In [8]:
# Testando o solucionador
initial_state = [[3,2], [0,1]]
goal_state = [[0,1],[2,3]]
puzzle = SlidingPuzzle(initial_state)
solver = BfsSolver(puzzle, goal_state)
solution = solver.solve()

if solution:
    print("Solução encontrada:")
    print(solution)
else:
    print("Não foi possível encontrar uma solução.")

Solução encontrada:
['up', 'right', 'down', 'left', 'up']


In [9]:
# Desempenho comparativo
import time

def pritnSolutionAndTime(s, t):
    if s:
        print("\tSolução encontrada:")
        print(f"\t{s}")
    else:
        print("\tNão foi possível encontrar uma solução.")
        
    print(f"\ttime: {t} s")


initial_state = [[3,2], [0,1]]
goal_state = [[0,1],[2,3]]


print("Backtrack")
print(f"\tinitial_state: {initial_state}")

t0 = time.perf_counter()

puzzle = SlidingPuzzle(initial_state)
solver = BacktrackingSolver(puzzle, goal_state)
solution = solver.solve()

t1 = time.perf_counter()
pritnSolutionAndTime(solution, t1-t0)


print("DFS")
print(f"\tinitial_state: {initial_state}")

t0 = time.perf_counter()

puzzle = SlidingPuzzle(initial_state)
solver = DfsSolver(puzzle, goal_state)
solution = solver.solve()

t1 = time.perf_counter()
pritnSolutionAndTime(solution, t1-t0)


print("BFS")
print(f"\tinitial_state: {initial_state}")

t0 = time.perf_counter()

puzzle = SlidingPuzzle(initial_state)
solver = BfsSolver(puzzle, goal_state)
solution = solver.solve()

t1 = time.perf_counter()
pritnSolutionAndTime(solution, t1-t0)

Backtrack
	initial_state: [[3, 2], [0, 1]]
	Solução encontrada:
	['up', 'right', 'down', 'left', 'up']
	time: 0.00018879999697674066 s
DFS
	initial_state: [[3, 2], [0, 1]]
	Solução encontrada:
	['up', 'right', 'down', 'left', 'up']
	time: 0.00018990000535268337 s
BFS
	initial_state: [[3, 2], [0, 1]]
	Solução encontrada:
	['up', 'right', 'down', 'left', 'up']
	time: 0.00048650000826455653 s


Utilizando as implementação apresentadas acima, nos casos do Backtrack e DFS, a solução para problemas com $N>2$ se torna inviável devido ao nível de profundidade na recursão. A seguir reimplementamos as soluçõe sem o uso da recursão.

#### Backtrack sem recursão

In [10]:
# Backtrack sem utilizar recursão
class BacktrackingSolver:
    def __init__(self, puzzle, goal):
        self.puzzle = puzzle
        self.visited = set()
        self.goal = goal

    def solve(self):
        # A solução inicializa com uma lsista vazia de movimentos
        solution = self.backtrack()
        return solution

    # Função de busca modificada para usar uma pilha em vez de recursão
    def backtrack(self):
        stack = [(self.puzzle.get_state(), 'none', [])] # Estado, movimento anterior, caminho até agora
        
        while stack:
            state, last_move, moves = stack.pop()
            self.puzzle.set_state(state)
            
            if self.is_goal(state):
                return moves
            
            for move in self.puzzle.get_possible_moves(last_move):
                self.puzzle.move(move)
                newState = self.puzzle.get_state()
                
                # Evita revisitar estados
                if newState not in self.visited:
                    self.visited.add(newState)
                    # Adiciona o novo estado, movimento e caminho à pilha
                    stack.append((newState, move, moves + [move]))
                
                # Desfaz o movimento para explorar outras possibilidades
                self.puzzle.undo_move(move)

        return None

    # Função verifica estado final
    def is_goal(self, state):
        state_list = [list(row) for row in state]
        return state_list == self.goal

#### DFS sem recursão

In [11]:
# DFS sem utilizar recursão
class DfsSolver:
    def __init__(self, puzzle, goal):
        self.puzzle = puzzle
        self.visited = set()
        self.goal = goal

    def solve(self):
        initial_state = self.puzzle.get_state()
        stack = [(initial_state, [])] # (estado, caminho até agora)

        while stack:
            state, path = stack.pop() # Obtem o último estado e caminho

            if self.is_goal(state):
                return path

            self.visited.add(state)
            self.puzzle.set_state(state)

            for move in self.puzzle.get_possible_moves():
                self.puzzle.move(move)
                next_state = self.puzzle.get_state()

                if next_state not in self.visited:
                    # Adiciona o estado seguinte e o novo caminho à pilha
                    new_path = path + [move]
                    stack.append((next_state, new_path))

                # Desfaz o movimento para tentar a próxima possibilidade
                self.puzzle.undo_move(move)

        return None

    # Função verifica estado final
    def is_goal(self, state):
        state_list = [list(row) for row in state]
        return state_list == self.goal

#### Busca de Custo Uniforme
A Busca de Custo Uniforme (também conhecida como busca de menor custo primeiro) é um algoritmo de busca não informada que prioriza a expansão do nó com o menor custo de caminho acumulado desde o nó inicial até o nó atual

In [12]:
import heapq  # Módulo para fila de prioridades

# Busca de Custo Uniforme sem utilizar recursão
class UniformCostSearchSolver:
    def __init__(self, puzzle, goal):
        self.puzzle = puzzle
        self.visited = set()
        self.goal = goal

    def solve(self):
        initial_state = self.puzzle.get_state()
        # A fila de prioridades contém elementos no formato: (custo, estado, caminho)
        priority_queue = [(0, initial_state, [])]  # (custo total até agora, estado, caminho até agora)

        while priority_queue:
            # Obtém o estado com o menor custo total até agora
            cost, state, path = heapq.heappop(priority_queue)

            if self.is_goal(state):
                return path

            if state not in self.visited:
                self.visited.add(state)
                self.puzzle.set_state(state)

                for move in self.puzzle.get_possible_moves():
                    self.puzzle.move(move)
                    next_state = self.puzzle.get_state()

                    if next_state not in self.visited:
                        # Calcula o novo custo como a soma do custo até o estado atual mais o custo do movimento
                        new_cost = cost + 1
                        new_path = path + [move]
                        # Adiciona o estado seguinte, o novo custo e o novo caminho à fila de prioridades
                        heapq.heappush(priority_queue, (new_cost, next_state, new_path))

                    # Desfaz o movimento para tentar a próxima possibilidade
                    self.puzzle.undo_move(move)

        return None

    # Função verifica estado final
    def is_goal(self, state):
        state_list = [list(row) for row in state]
        return state_list == self.goal

In [13]:
# Desempenho comparativo
import time

def pritnSolutionAndTime(s, t):
    if s:
        print("\tSolução encontrada:")
        print(f"\t{s}")
    else:
        print("\tNão foi possível encontrar uma solução.")
        
    print(f"\ttime: {t} s")


initial_state = [[2,8,3], [1,6,4], [7,0,5]]
goal_state = [[1,2,3],[8,0,4],[7,6,5]]


print("Backtrack")
print(f"\tinitial_state: {initial_state}")

t0 = time.perf_counter()

puzzle = SlidingPuzzle(initial_state)
solver = BacktrackingSolver(puzzle, goal_state)
solution = solver.solve()

t1 = time.perf_counter()
pritnSolutionAndTime(solution, t1-t0)


print("DFS")
print(f"\tinitial_state: {initial_state}")

t0 = time.perf_counter()

puzzle = SlidingPuzzle(initial_state)
solver = DfsSolver(puzzle, goal_state)
solution = solver.solve()

t1 = time.perf_counter()
pritnSolutionAndTime(solution, t1-t0)


print("BFS")
print(f"\tinitial_state: {initial_state}")

t0 = time.perf_counter()

puzzle = SlidingPuzzle(initial_state)
solver = BfsSolver(puzzle, goal_state)
solution = solver.solve()

t1 = time.perf_counter()
pritnSolutionAndTime(solution, t1-t0)


print("UCS")
print(f"\tinitial_state: {initial_state}")

t0 = time.perf_counter()

puzzle = SlidingPuzzle(initial_state)
solver = UniformCostSearchSolver(puzzle, goal_state)
solution = solver.solve()

t1 = time.perf_counter()
pritnSolutionAndTime(solution, t1-t0)

Backtrack
	initial_state: [[2, 8, 3], [1, 6, 4], [7, 0, 5]]
	Solução encontrada:
	['right', 'up', 'left', 'left', 'down', 'right', 'right', 'up', 'left', 'left', 'down', 'right', 'right', 'up', 'left', 'left', 'down', 'right', 'right', 'up', 'left', 'left', 'down', 'right', 'right', 'up', 'left', 'left', 'up', 'right', 'right', 'down', 'left', 'left', 'down', 'right', 'right', 'up', 'left', 'left', 'down', 'right', 'right', 'up', 'left', 'left', 'down', 'right', 'right', 'up', 'left', 'left', 'down', 'right', 'right', 'up', 'left', 'left', 'down', 'right', 'up', 'right', 'down', 'left', 'left', 'up', 'right', 'right', 'down', 'left', 'left', 'up', 'right', 'right', 'up', 'left', 'left', 'down', 'right', 'right', 'down', 'left', 'left', 'up', 'right', 'right', 'down', 'left', 'left', 'up', 'right', 'right', 'down', 'left', 'left', 'up', 'right', 'right', 'down', 'left', 'left', 'up', 'right', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'left', 'up', 'right', 'right', 'down',

## Busca Heurística

#### Função de Avaliação
Definimos a seguir a distância de Manhattan e a distância de Hamming para Sliding Puzzle. Estas duas são utilizadas como função de avaliação nos algoritmos de busca com heurística implementados a seguir.

In [14]:
def manhattan_distance(state, goal):
    """
    Calcula a distância de Manhattan total para o estado atual em relação ao estado objetivo.
    """
    total_distance = 0
    for i, row in enumerate(state):
        for j, value in enumerate(row):
            if value == 0:  # Ignora o espaço vazio
                continue
            # Encontra a posição de 'value' no estado objetivo
            goal_i, goal_j = next((x, y) for x, row in enumerate(goal) for y, v in enumerate(row) if v == value)
            # Soma a distância de Manhattan para este 'value'
            total_distance += abs(i - goal_i) + abs(j - goal_j)
    return total_distance

In [15]:
def hamming_distance(state, goal):
    """
    Calcula a distância de Hamming para o estado atual em relação ao estado objetivo.
    """
    distance = 0
    for i, row in enumerate(state):
        for j, value in enumerate(row):
            if value == 0:  # Ignora o espaço vazio
                continue
            if goal[i][j] != value:  # Conta uma peça fora do lugar
                distance += 1
    return distance

In [16]:
# Teste das funções distância
goal_state = [[1,2,3],[8,0,4],[7,6,5]]

state = [[2,8,3], [1,6,4], [7,0,5]]
d = manhattan_distance(state, goal_state)
print(f"manhattan_distance = {d}")
d = hamming_distance(state, goal_state)
print(f"hamming_distance = {d}")

state = [[5,8,3], [1,6,4], [7,0,2]]
d = manhattan_distance(state, goal_state)
print(f"manhattan_distance = {d}")
d = hamming_distance(state, goal_state)
print(f"hamming_distance = {d}")

manhattan_distance = 5
hamming_distance = 4
manhattan_distance = 11
hamming_distance = 5


#### Busca Gulosa

In [17]:
import heapq  # Módulo para fila de prioridades

class GreedySolver:
    def __init__(self, puzzle, goal, heuristic):
        self.puzzle = puzzle
        self.visited = set()
        self.goal = goal
        self.heuristic = heuristic  # Função heurística

    def solve(self):
        initial_state = self.puzzle.get_state()
        # A fila de prioridades contém elementos no formato: (heurística, estado, caminho)
        priority_queue = [(self.heuristic(initial_state), initial_state, [])]  # (heurística do estado, estado, caminho até agora)

        while priority_queue:
            # Obtém o estado com a menor estimativa heurística
            estimated_cost, state, path = heapq.heappop(priority_queue)

            if self.is_goal(state):
                return path

            if state not in self.visited:
                self.visited.add(state)
                self.puzzle.set_state(state)

                for move in self.puzzle.get_possible_moves():
                    self.puzzle.move(move)
                    next_state = self.puzzle.get_state()

                    if next_state not in self.visited:
                        # Atualiza a estimativa heurística para o novo estado
                        new_estimated_cost = self.heuristic(next_state)
                        new_path = path + [move]
                        # Adiciona o estado seguinte e a nova estimativa à fila de prioridades
                        heapq.heappush(priority_queue, (new_estimated_cost, next_state, new_path))

                    # Desfaz o movimento para tentar a próxima possibilidade
                    self.puzzle.undo_move(move)

        return None

    # Função verifica estado final
    def is_goal(self, state):
        state_list = [list(row) for row in state]
        return state_list == self.goal

In [18]:
initial_state = [[2,8,3], [1,6,4], [7,0,5]]
goal_state = [[1,2,3],[8,0,4],[7,6,5]]

print("Greedy h: Manhattan")
print(f"\tinitial_state: {initial_state}")

t0 = time.perf_counter()

puzzle = SlidingPuzzle(initial_state)
solver = GreedySolver(puzzle, goal_state, lambda state: manhattan_distance(state, goal_state))
solution = solver.solve()

t1 = time.perf_counter()
pritnSolutionAndTime(solution, t1-t0)

print("Greedy h: Hamming")
print(f"\tinitial_state: {initial_state}")

t0 = time.perf_counter()

puzzle = SlidingPuzzle(initial_state)
solver = GreedySolver(puzzle, goal_state, lambda state: hamming_distance(state, goal_state))
solution = solver.solve()

t1 = time.perf_counter()
pritnSolutionAndTime(solution, t1-t0)

Greedy h: Manhattan
	initial_state: [[2, 8, 3], [1, 6, 4], [7, 0, 5]]
	Solução encontrada:
	['up', 'up', 'left', 'down', 'right']
	time: 0.0008728999964660034 s
Greedy h: Hamming
	initial_state: [[2, 8, 3], [1, 6, 4], [7, 0, 5]]
	Solução encontrada:
	['up', 'up', 'left', 'down', 'right']
	time: 0.0003750999894691631 s


#### Busca com A*

In [19]:
import heapq

class AStarSolver:
    def __init__(self, puzzle, goal, heuristic):
        self.puzzle = puzzle
        self.goal = goal
        self.heuristic = heuristic

    def solve(self):
        initial_state = self.puzzle.get_state()
        # A fila de prioridades armazena elementos no formato: (f(n), g(n), estado, caminho)
        priority_queue = [(self.heuristic(initial_state), 0, initial_state, [])]
        visited = set()

        while priority_queue:
            f, g, state, path = heapq.heappop(priority_queue)

            if self.is_goal(state):
                return path

            state_repr = self.state_representation(state)
            if state_repr in visited:
                continue
            visited.add(state_repr)

            self.puzzle.set_state(state)

            for move in self.puzzle.get_possible_moves():
                self.puzzle.move(move)
                next_state = self.puzzle.get_state()
                next_state_repr = self.state_representation(next_state)
                
                if next_state_repr not in visited:
                    new_g = g + 1  # Assumindo custo uniforme de 1 para cada movimento
                    new_h = self.heuristic(next_state)
                    new_f = new_g + new_h
                    new_path = path + [move]
                    heapq.heappush(priority_queue, (new_f, new_g, next_state, new_path))

                self.puzzle.undo_move(move)
        
        return None

    def is_goal(self, state):
        state_list = [list(row) for row in state]
        return state_list == self.goal
    
    def state_representation(self, state):
        """Converte o estado em uma representação imutável para armazenamento."""
        return tuple(tuple(row) for row in state)

In [20]:
initial_state = [[2,8,3], [1,6,4], [7,0,5]]
goal_state = [[1,2,3],[8,0,4],[7,6,5]]

print("A* h: Manhattan")
print(f"\tinitial_state: {initial_state}")

t0 = time.perf_counter()

puzzle = SlidingPuzzle(initial_state)
solver = AStarSolver(puzzle, goal_state, lambda state: manhattan_distance(state, goal_state))
solution = solver.solve()

t1 = time.perf_counter()
pritnSolutionAndTime(solution, t1-t0)

print("A* h: Hamming")
print(f"\tinitial_state: {initial_state}")

t0 = time.perf_counter()

puzzle = SlidingPuzzle(initial_state)
solver = AStarSolver(puzzle, goal_state, lambda state: hamming_distance(state, goal_state))
solution = solver.solve()

t1 = time.perf_counter()
pritnSolutionAndTime(solution, t1-t0)

A* h: Manhattan
	initial_state: [[2, 8, 3], [1, 6, 4], [7, 0, 5]]
	Solução encontrada:
	['up', 'up', 'left', 'down', 'right']
	time: 0.000861099993926473 s
A* h: Hamming
	initial_state: [[2, 8, 3], [1, 6, 4], [7, 0, 5]]
	Solução encontrada:
	['up', 'up', 'left', 'down', 'right']
	time: 0.000571600001421757 s


## Metaheurísticas

#### Simulated Annealing

In [ ]:
import random
import math

class SimulatedAnnealingSolver:
    def __init__(self, puzzle, goal, temperature=100, cooling_rate=0.99, steps_per_temp=100):
        self.puzzle = puzzle
        self.goal = goal
        self.temperature = temperature
        self.cooling_rate = cooling_rate
        self.steps_per_temp = steps_per_temp

    def heuristic(self, state):
        """Uma função heurística simples que conta o número de peças fora do lugar."""
        distance = 0
        for i in range(len(state)):
            for j in range(len(state[i])):
                if state[i][j] != 0 and state[i][j] != self.goal[i][j]:
                    distance += 1
        return distance

    def solve(self):
        current_state = self.puzzle.get_state()
        current_cost = self.heuristic(current_state)

        while self.temperature > 1:
            for _ in range(self.steps_per_temp):
                possible_moves = self.puzzle.get_possible_moves()
                if not possible_moves:
                    continue

                move = random.choice(possible_moves)
                self.puzzle.move(move)
                next_state = self.puzzle.get_state()
                next_cost = self.heuristic(next_state)

                cost_diff = next_cost - current_cost

                if cost_diff < 0 or math.exp(-cost_diff / self.temperature) > random.random():
                    current_state = next_state
                    current_cost = next_cost
                    if current_cost == 0:  # Solução encontrada
                        return True
                else:
                    self.puzzle.undo_move(move)  # Desfaz o movimento se não for aceito

            self.temperature *= self.cooling_rate  # Reduz a temperatura

        return False  # Retorna False se a solução não for encontrada

In [ ]:
initial_state = [[2,8,3], [1,6,4], [7,0,5]]
goal_state = [[1,2,3],[8,0,4],[7,6,5]]

puzzle = SlidingPuzzle(initial_state)
solver = SimulatedAnnealingSolver(puzzle, goal_state)
if solver.solve():
    print("Solução encontrada!")
else:
    print("Solução não encontrada.")
